In [1]:
import torch
from salsaclrs import load_dataset

2023-11-30 15:51:50.191067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 15:51:50.191095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 15:51:50.192273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 15:51:51.157905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/vgauthier/.conda/envs/clrs/lib/python3.10/site-packages/torch_geometric/typing.py:63: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: libcudart.so.11.0: cannot open shared object file: N

In [2]:
train_dataset = load_dataset(algorithm="bfs", split="train", local_dir="./store")

In [3]:
data = train_dataset[0]

In [4]:
from salsaclrs import SALSACLRSDataLoader
dl = SALSACLRSDataLoader(train_dataset, batch_size=1)

In [5]:
it = iter(dl)

In [6]:
batch = next(it)

In [7]:
batch

CLRSDataBatch(edge_index=[2, 10], pos=[4], length=[1], s=[4], pi=[10], reach_h=[4, 4], pi_h=[10, 4], hints=[2], inputs=[2], outputs=[1], batch=[4], ptr=[2])

In [8]:
from models import Encoder
from models import Processor
from models import EdgeMaskDecoder

In [26]:
data.length.item() == 3 

False

In [10]:
hidden_dim = 128
enc = Encoder(hidden_dim=hidden_dim)
proc_input_dim = 3 * hidden_dim +1 
proc = Processor(proc_input_dim, hidden_dim, aggr="max")
dec = EdgeMaskDecoder(hidden_dim)

In [12]:
pos = data.pos
x = data.s
edge_index = data.edge_index

In [13]:
input_hidden = enc(x)
hidden = input_hidden
last_hidden = hidden

In [14]:
hidden = proc(input_hidden, input_hidden, last_hidden, edge_index, pos)

In [48]:
output_step = dec(hidden, edge_index)

In [49]:
for k in output_step:
    print(k)

tensor(0.4346, grad_fn=<UnbindBackward0>)
tensor(0.3537, grad_fn=<UnbindBackward0>)
tensor(0.4946, grad_fn=<UnbindBackward0>)
tensor(0.3746, grad_fn=<UnbindBackward0>)
tensor(0.2951, grad_fn=<UnbindBackward0>)
tensor(0.4433, grad_fn=<UnbindBackward0>)
tensor(0.5072, grad_fn=<UnbindBackward0>)
tensor(0.5086, grad_fn=<UnbindBackward0>)
tensor(0.5087, grad_fn=<UnbindBackward0>)
tensor(0.5102, grad_fn=<UnbindBackward0>)


In [30]:
batch.edge_attrs()

['pi_h', 'edge_index', 'pi']

In [31]:
batch.length[batch.batch]-1

tensor([3, 3, 3, 3])

In [32]:
batch.batch

tensor([0, 0, 0, 0])

In [33]:
batch.length

tensor([4])

In [34]:
final_node_idx = (batch.length[batch.batch]-1)

In [35]:
final_node_idx

tensor([3, 3, 3, 3])

In [38]:
batch.edge_index[0]

tensor([0, 0, 0, 1, 1, 2, 2, 2, 3, 3])

In [50]:
masks = {}
step = 3
for key in batch.outputs:
    final_edge_idx = final_node_idx[batch.edge_index[0]]
    masks[key] = final_edge_idx == step

In [51]:
final_edge_idx

tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [52]:
masks

{'pi': tensor([True, True, True, True, True, True, True, True, True, True])}